In [16]:
import numpy as np
import pickle
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
import load
from sklearn.metrics import classification_report

In [17]:
wesad = load.WESAD()
df = wesad.separate_and_feature_extract(sample_n=420000) #700 * 60 * 10 (10mins)

Using ready-made DataFrame


In [18]:
df.shape

(30, 59, 20)

In [19]:
y = df[:, 0, 0]  # 取得 'label'
X = df[:, :, 2:]  # 移除 'label','subject'

In [20]:
y=y-1

In [21]:
import numpy as np

X = np.array(X, dtype=np.float32)

y = np.array(y, dtype=np.int32)

print("X dtype:", X.dtype)  # 應該是 float32
print("y dtype:", y.dtype)  # 應該是 int32

X dtype: float32
y dtype: int32


In [22]:
# 分割訓練測試集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [23]:
def build_cnn_model(input_shape, num_classes):
    model = models.Sequential()
    model.add(layers.Conv1D(filters=32, kernel_size=5, activation='relu', padding='same', input_shape=input_shape))
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.Conv1D(filters=64, kernel_size=5, activation='relu', padding='same'))
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.Conv1D(filters=128, kernel_size=3, activation='relu', padding='same'))
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(num_classes, activation='softmax'))
    
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

In [24]:
def build_cnn_lstm_model(input_shape, num_classes):
    model = models.Sequential()
    model.add(layers.Conv1D(filters=32, kernel_size=5, activation='relu', padding='same', input_shape=input_shape))
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.Conv1D(filters=64, kernel_size=5, activation='relu', padding='same'))
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.Conv1D(filters=128, kernel_size=3, activation='relu', padding='same'))
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.LSTM(128, return_sequences=True))
    model.add(layers.LSTM(64))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(num_classes, activation='softmax'))
    
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

In [25]:
input_shape = (X_train.shape[1], X_train.shape[2]) #(59,18)
num_classes = len(np.unique(y_train))

print(input_shape)
print(num_classes)

(59, 18)
2


In [26]:
# 初始化模型
model = build_cnn_model(input_shape, num_classes)

# 編譯模型
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 訓練模型
epochs = 30
batch_size = 16
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test))

Epoch 1/30


c:\tensorflow\.venv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 145ms/step - accuracy: 0.4633 - loss: 2.3110 - val_accuracy: 0.3333 - val_loss: 3.2337
Epoch 2/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.5893 - loss: 1.7755 - val_accuracy: 0.6667 - val_loss: 0.6746
Epoch 3/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.4524 - loss: 0.8348 - val_accuracy: 0.6667 - val_loss: 0.8010
Epoch 4/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.4524 - loss: 0.9919 - val_accuracy: 0.3333 - val_loss: 0.7515
Epoch 5/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.7470 - loss: 0.6453 - val_accuracy: 0.3333 - val_loss: 1.1822
Epoch 6/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.5476 - loss: 0.8312 - val_accuracy: 0.4444 - val_loss: 1.0307
Epoch 7/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.6101 - loss: 0.6236 - val_accuracy: 0.4444 - val_loss: 0.7230
Epoch 8/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.6210 - loss: 0.6208 - val_accuracy: 0.7778 - val_loss: 0.6238
Epoch 9/30

In [27]:
y_pred_prob = model.predict(X_test)

# 將機率轉換為 0 或 1（閾值設為 0.5）
y_pred_classes = (y_pred_prob > 0.5).astype(int)

y_pred_single = np.argmax(y_pred_classes, axis=1)

# 輸出分類報告
report = classification_report(y_test, y_pred_single, target_names=['1', '2'])
print(report)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
              precision    recall  f1-score   support

           1       0.25      0.33      0.29         3
           2       0.60      0.50      0.55         6

    accuracy                           0.44         9
   macro avg       0.42      0.42      0.42         9
weighted avg       0.48      0.44      0.46         9



In [28]:
# 初始化模型
model = build_cnn_lstm_model(input_shape, num_classes)

# 編譯模型
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 訓練模型
epochs = 30
batch_size = 16
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test))

Epoch 1/30


c:\tensorflow\.venv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 303ms/step - accuracy: 0.5159 - loss: 0.6980 - val_accuracy: 0.3333 - val_loss: 0.8270
Epoch 2/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.5685 - loss: 0.6928 - val_accuracy: 0.3333 - val_loss: 0.7870
Epoch 3/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.5476 - loss: 0.7036 - val_accuracy: 0.3333 - val_loss: 0.7556
Epoch 4/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.5476 - loss: 0.6812 - val_accuracy: 0.3333 - val_loss: 0.8380
Epoch 5/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.5685 - loss: 0.6908 - val_accuracy: 0.3333 - val_loss: 0.8607
Epoch 6/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.5476 - loss: 0.7108 - val_accuracy: 0.3333 - val_loss: 0.7899
Epoch 7/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.5476 - loss: 0.6815 - val_accuracy: 0.3333 - val_loss: 0.7907
Epoch 8/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.5476 - loss: 0.6693 - val_accuracy: 0.4444 - val_loss: 0.8344
Epoch 9/30

In [29]:
y_pred_prob = model.predict(X_test)

# 將機率轉換為 0 或 1（閾值設為 0.5）
y_pred_classes = (y_pred_prob > 0.5).astype(int)

y_pred_single = np.argmax(y_pred_classes, axis=1)

# 輸出分類報告
report = classification_report(y_test, y_pred_single, target_names=['1', '2'])
print(report)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 204ms/step
              precision    recall  f1-score   support

           1       0.50      0.33      0.40         3
           2       0.71      0.83      0.77         6

    accuracy                           0.67         9
   macro avg       0.61      0.58      0.58         9
weighted avg       0.64      0.67      0.65         9



In [30]:
# 選擇 LSTM 是因為 生理訊號屬於時間序列數據，LSTM 能夠記住過去的資訊，並利用這些資訊影響未來的預測。
# 此外，LSTM 透過 記憶單元與門機制，有效解決了傳統 RNN 中的 梯度消失問題，使模型能夠學習並捕捉長時間的依賴關係，從而提升壓力偵測的準確性。